# Vehicle Detection Project

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import cv2, glob, time
import numpy as np

from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from skimage.feature import hog

%matplotlib inline

## Features extraction
The functions below are based on source codes from the Udacity's lectures.

In [2]:
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    if vis == True: # Call with two outputs if vis==True to visualize the HOG
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    else:      # Otherwise call with one output
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

# Define a function to compute binned color features  
def bin_spatial(img, size=(16, 16)):
    return cv2.resize(img, size).ravel() 

# Define a function to compute color histogram features 
def color_hist(img, nbins=32):
    ch1 = np.histogram(img[:,:,0], bins=nbins, range=(0, 256))[0]#We need only the histogram, no bins edges
    ch2 = np.histogram(img[:,:,1], bins=nbins, range=(0, 256))[0]
    ch3 = np.histogram(img[:,:,2], bins=nbins, range=(0, 256))[0]
    hist = np.hstack((ch1, ch2, ch3))
    return hist

def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

In [3]:
def augment_image(image):
    images = []
    images.append(image)
    images.append(cv2.flip(image,1))
    return images

def img_features(feature_image, spatial_feat, hist_feat, hog_feat, hist_bins, orient, 
                        pix_per_cell, cell_per_block, hog_channel):
    file_features = []
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        file_features.append(spatial_features)
    if hist_feat == True:
         # Apply color_hist()
        hist_features = color_hist(feature_image, nbins=hist_bins)
        file_features.append(hist_features)
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
        else:
            feature_image = cv2.cvtColor(feature_image, cv2.COLOR_LUV2RGB)
            feature_image = cv2.cvtColor(feature_image, cv2.COLOR_RGB2GRAY)
            hog_features = get_hog_features(feature_image[:,:], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
        file_features.append(hog_features)
    return file_features

def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file_p in imgs:
        file_features = []
        image = cv2.imread(file_p)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        augment_images = augment_image(feature_image) 
        for aug_img in augment_images:
            file_features = img_features(aug_img, spatial_feat, hist_feat, hog_feat, hist_bins, orient, 
                            pix_per_cell, cell_per_block, hog_channel)
            features.append(np.concatenate(file_features))

    return features # Return list of feature vectors

## Train SVC Classfier

In [5]:
# Read in cars and notcars
images = glob.glob('*vehicles/*/*')
cars = []
not_cars = []
for image in images:
    if 'non' in image:
        not_cars.append(image)
    else:
        cars.append(image)
        
print('Number of Car samples:', len(cars))
print('Number of Not Car samples:', len(not_cars))

Number of Car samples: 8792
Number of Not Car samples: 8968


In [8]:
color_space = 'LUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 8  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 0 # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

print('Done car_features extraction')

not_car_features = extract_features(not_cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

print('Done not_car_features extraction')

X = np.vstack((car_features, not_car_features)).astype(np.float64)                        

X_scaler = StandardScaler().fit(X) # Fit a per-column scaler
scaled_X = X_scaler.transform(X) # Apply the scaler to X

y = np.hstack((np.ones(len(car_features)), np.zeros(len(not_car_features)))) # Define the labels vector

# Split up data into randomized training and test sets
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=22)

svc = LinearSVC(C=0.01) # Use a linear SVC 
svc.fit(X_train, y_train) # Train the classifier

print('Test Accuracy is', svc.score(X_test, y_test))

/Users/pchy617h/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


Done car_features extraction
Done not_car_features extraction
Test Accuracy is 0.9923464038582512


## Classifier Test with Sliding Window

In [9]:
# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)


def extract_features_by_window(img, window, scaler):
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        #4) Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        
        return test_features
    
# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image

        test_features = extract_features_by_window(img, window, scaler)
        
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows

## Advanced Sliding Windows
Reduce false positive and duplicate detections with heatmap approach

In [85]:
from scipy.ndimage.measurements import label

def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):

    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
            
        test_features = extract_features_by_window(img, bbox, X_scaler)
        
        score = svc.decision_function(test_features)[0]
        #print (score)
        if score > 0.7:
            cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)

#         prediction = svc.predict(test_features)
#         if prediction == 1:
#             cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
            
    # Return the image
    return img


frame_counter = 0
heat = np.zeros((720, 1280)).astype(np.float)

def find_car(org_image):
    global frame_counter, heat
    frame_counter += 1
    
    image = cv2.cvtColor(org_image, cv2.COLOR_RGB2BGR)
    
    windows = slide_window(image, x_start_stop=[None, None], y_start_stop=[400, 640], 
                    xy_window=(96, 96), xy_overlap=(0.85, 0.85))
    
   
    box_list = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)  

    # Add heat to each box in box list
    heat = add_heat(heat,box_list)
    
    if frame_counter < 12:
        return org_image

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,1)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)
    
    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    
    draw_img = draw_labeled_bboxes(np.copy(org_image), labels)
    
    frame_counter = 0
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    
    return draw_img

## Video processing

In [86]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def process_image(org_image):
    return find_car(org_image)

video_out = 'video_output/project_video.mp4'
myclip = VideoFileClip('project_video.mp4')
clip = myclip.fl_image(process_image)

clip.write_videofile(video_out, audio=False)

[MoviePy] >>>> Building video video_output/project_video.mp4
[MoviePy] Writing video video_output/project_video.mp4





  0%|          | 0/1261 [00:00<?, ?it/s]


  0%|          | 1/1261 [00:01<41:35,  1.98s/it]


  0%|          | 2/1261 [00:03<41:25,  1.97s/it]


  0%|          | 3/1261 [00:05<41:17,  1.97s/it]


  0%|          | 4/1261 [00:07<41:21,  1.97s/it]


  0%|          | 5/1261 [00:09<41:14,  1.97s/it]


  0%|          | 6/1261 [00:11<40:59,  1.96s/it]


  1%|          | 7/1261 [00:13<40:47,  1.95s/it]


  1%|          | 8/1261 [00:15<40:39,  1.95s/it]


  1%|          | 9/1261 [00:17<40:48,  1.96s/it]


  1%|          | 10/1261 [00:19<40:41,  1.95s/it]


  1%|          | 11/1261 [00:21<40:43,  1.95s/it]


  1%|          | 12/1261 [00:23<40:47,  1.96s/it]


  1%|          | 13/1261 [00:25<40:37,  1.95s/it]


  1%|          | 14/1261 [00:27<41:07,  1.98s/it]


  1%|          | 15/1261 [00:29<40:57,  1.97s/it]


  1%|▏         | 16/1261 [00:31<40:43,  1.96s/it]


  1%|▏         | 17/1261 [00:33<40:27,  1.95s/it]


  1%|▏         | 18/1261 [00:35<40:11,  1.94s/it]


  2%|▏         | 19/1261 [0

 11%|█         | 134/1261 [04:18<36:03,  1.92s/it]


 11%|█         | 135/1261 [04:20<36:29,  1.94s/it]


 11%|█         | 136/1261 [04:22<36:26,  1.94s/it]


 11%|█         | 137/1261 [04:24<36:17,  1.94s/it]


 11%|█         | 138/1261 [04:26<38:21,  2.05s/it]


 11%|█         | 139/1261 [04:28<39:53,  2.13s/it]


 11%|█         | 140/1261 [04:31<40:45,  2.18s/it]


 11%|█         | 141/1261 [04:33<41:21,  2.22s/it]


 11%|█▏        | 142/1261 [04:35<41:02,  2.20s/it]


 11%|█▏        | 143/1261 [04:37<39:32,  2.12s/it]


 11%|█▏        | 144/1261 [04:39<39:51,  2.14s/it]


 11%|█▏        | 145/1261 [04:42<41:01,  2.21s/it]


 12%|█▏        | 146/1261 [04:44<41:23,  2.23s/it]


 12%|█▏        | 147/1261 [04:46<41:38,  2.24s/it]


 12%|█▏        | 148/1261 [04:48<41:26,  2.23s/it]


 12%|█▏        | 149/1261 [04:51<41:33,  2.24s/it]


 12%|█▏        | 150/1261 [04:53<40:38,  2.19s/it]


 12%|█▏        | 151/1261 [04:55<39:55,  2.16s/it]


 12%|█▏        | 152/1261 [04:57<39:36,  2.14s

 21%|██        | 266/1261 [08:39<31:59,  1.93s/it]


 21%|██        | 267/1261 [08:41<31:56,  1.93s/it]


 21%|██▏       | 268/1261 [08:43<31:47,  1.92s/it]


 21%|██▏       | 269/1261 [08:45<31:49,  1.92s/it]


 21%|██▏       | 270/1261 [08:47<31:46,  1.92s/it]


 21%|██▏       | 271/1261 [08:49<31:46,  1.93s/it]


 22%|██▏       | 272/1261 [08:51<31:42,  1.92s/it]


 22%|██▏       | 273/1261 [08:52<31:32,  1.92s/it]


 22%|██▏       | 274/1261 [08:54<31:28,  1.91s/it]


 22%|██▏       | 275/1261 [08:56<31:28,  1.92s/it]


 22%|██▏       | 276/1261 [08:58<31:25,  1.91s/it]


 22%|██▏       | 277/1261 [09:00<31:15,  1.91s/it]


 22%|██▏       | 278/1261 [09:02<31:11,  1.90s/it]


 22%|██▏       | 279/1261 [09:04<31:06,  1.90s/it]


 22%|██▏       | 280/1261 [09:06<31:36,  1.93s/it]


 22%|██▏       | 281/1261 [09:08<31:40,  1.94s/it]


 22%|██▏       | 282/1261 [09:10<31:27,  1.93s/it]


 22%|██▏       | 283/1261 [09:12<31:16,  1.92s/it]


 23%|██▎       | 284/1261 [09:14<31:11,  1.92s

 32%|███▏      | 398/1261 [12:53<27:30,  1.91s/it]


 32%|███▏      | 399/1261 [12:54<27:30,  1.91s/it]


 32%|███▏      | 400/1261 [12:56<27:23,  1.91s/it]


 32%|███▏      | 401/1261 [12:58<27:16,  1.90s/it]


 32%|███▏      | 402/1261 [13:00<27:15,  1.90s/it]


 32%|███▏      | 403/1261 [13:02<27:09,  1.90s/it]


 32%|███▏      | 404/1261 [13:04<27:14,  1.91s/it]


 32%|███▏      | 405/1261 [13:06<27:07,  1.90s/it]


 32%|███▏      | 406/1261 [13:08<27:02,  1.90s/it]


 32%|███▏      | 407/1261 [13:10<27:06,  1.90s/it]


 32%|███▏      | 408/1261 [13:12<27:03,  1.90s/it]


 32%|███▏      | 409/1261 [13:13<27:01,  1.90s/it]


 33%|███▎      | 410/1261 [13:15<27:00,  1.90s/it]


 33%|███▎      | 411/1261 [13:17<26:55,  1.90s/it]


 33%|███▎      | 412/1261 [13:19<26:52,  1.90s/it]


 33%|███▎      | 413/1261 [13:21<26:51,  1.90s/it]


 33%|███▎      | 414/1261 [13:23<26:51,  1.90s/it]


 33%|███▎      | 415/1261 [13:25<26:47,  1.90s/it]


 33%|███▎      | 416/1261 [13:27<27:02,  1.92s

...

 63%|██████▎   | 794/1261 [25:33<14:55,  1.92s/it]


 63%|██████▎   | 795/1261 [25:35<14:57,  1.93s/it]


 63%|██████▎   | 796/1261 [25:37<14:51,  1.92s/it]


 63%|██████▎   | 797/1261 [25:39<14:46,  1.91s/it]


 63%|██████▎   | 798/1261 [25:41<14:42,  1.91s/it]


 63%|██████▎   | 799/1261 [25:43<14:41,  1.91s/it]


 63%|██████▎   | 800/1261 [25:44<14:38,  1.90s/it]


 64%|██████▎   | 801/1261 [25:46<14:36,  1.90s/it]


 64%|██████▎   | 802/1261 [25:48<14:36,  1.91s/it]


 64%|██████▎   | 803/1261 [25:50<14:37,  1.92s/it]


 64%|██████▍   | 804/1261 [25:52<14:33,  1.91s/it]


 64%|██████▍   | 805/1261 [25:54<14:30,  1.91s/it]


 64%|██████▍   | 806/1261 [25:56<14:29,  1.91s/it]


 64%|██████▍   | 807/1261 [25:58<14:27,  1.91s/it]


 64%|██████▍   | 808/1261 [26:00<14:25,  1.91s/it]


 64%|██████▍   | 809/1261 [26:02<14:24,  1.91s/it]


 64%|██████▍   | 810/1261 [26:04<14:20,  1.91s/it]


 64%|██████▍   | 811/1261 [26:06<14:20,  1.91s/it]


 64%|██████▍   | 812/1261 [26:07<14:20,  1.92s

 73%|███████▎  | 926/1261 [29:47<10:43,  1.92s/it]


 74%|███████▎  | 927/1261 [29:49<10:41,  1.92s/it]


 74%|███████▎  | 928/1261 [29:51<10:41,  1.93s/it]


 74%|███████▎  | 929/1261 [29:53<10:38,  1.92s/it]


 74%|███████▍  | 930/1261 [29:54<10:37,  1.93s/it]


 74%|███████▍  | 931/1261 [29:56<10:37,  1.93s/it]


 74%|███████▍  | 932/1261 [29:58<10:34,  1.93s/it]


 74%|███████▍  | 933/1261 [30:00<10:30,  1.92s/it]


 74%|███████▍  | 934/1261 [30:02<10:26,  1.92s/it]


 74%|███████▍  | 935/1261 [30:04<10:25,  1.92s/it]


 74%|███████▍  | 936/1261 [30:06<10:23,  1.92s/it]


 74%|███████▍  | 937/1261 [30:08<10:22,  1.92s/it]


 74%|███████▍  | 938/1261 [30:10<10:18,  1.92s/it]


 74%|███████▍  | 939/1261 [30:12<10:15,  1.91s/it]


 75%|███████▍  | 940/1261 [30:14<10:12,  1.91s/it]


 75%|███████▍  | 941/1261 [30:16<10:10,  1.91s/it]


 75%|███████▍  | 942/1261 [30:17<10:09,  1.91s/it]


 75%|███████▍  | 943/1261 [30:19<10:08,  1.91s/it]


 75%|███████▍  | 944/1261 [30:21<10:06,  1.91s

 84%|████████▍ | 1057/1261 [33:57<06:29,  1.91s/it]


 84%|████████▍ | 1058/1261 [33:59<06:27,  1.91s/it]


 84%|████████▍ | 1059/1261 [34:01<06:25,  1.91s/it]


 84%|████████▍ | 1060/1261 [34:03<06:22,  1.90s/it]


 84%|████████▍ | 1061/1261 [34:05<06:38,  1.99s/it]


 84%|████████▍ | 1062/1261 [34:07<06:31,  1.97s/it]


 84%|████████▍ | 1063/1261 [34:09<06:26,  1.95s/it]


 84%|████████▍ | 1064/1261 [34:11<06:21,  1.94s/it]


 84%|████████▍ | 1065/1261 [34:13<06:18,  1.93s/it]


 85%|████████▍ | 1066/1261 [34:15<06:16,  1.93s/it]


 85%|████████▍ | 1067/1261 [34:17<06:15,  1.94s/it]


 85%|████████▍ | 1068/1261 [34:19<06:12,  1.93s/it]


 85%|████████▍ | 1069/1261 [34:20<06:09,  1.93s/it]


 85%|████████▍ | 1070/1261 [34:22<06:07,  1.92s/it]


 85%|████████▍ | 1071/1261 [34:24<06:05,  1.92s/it]


 85%|████████▌ | 1072/1261 [34:26<06:06,  1.94s/it]


 85%|████████▌ | 1073/1261 [34:29<06:37,  2.12s/it]


 85%|████████▌ | 1074/1261 [34:31<06:46,  2.18s/it]


 85%|████████▌ | 1075/1261 [

 94%|█████████▍| 1187/1261 [38:09<02:22,  1.92s/it]


 94%|█████████▍| 1188/1261 [38:11<02:20,  1.92s/it]


 94%|█████████▍| 1189/1261 [38:12<02:17,  1.91s/it]


 94%|█████████▍| 1190/1261 [38:14<02:15,  1.91s/it]


 94%|█████████▍| 1191/1261 [38:16<02:13,  1.90s/it]


 95%|█████████▍| 1192/1261 [38:18<02:11,  1.91s/it]


 95%|█████████▍| 1193/1261 [38:20<02:10,  1.92s/it]


 95%|█████████▍| 1194/1261 [38:22<02:08,  1.92s/it]


 95%|█████████▍| 1195/1261 [38:24<02:06,  1.91s/it]


 95%|█████████▍| 1196/1261 [38:26<02:03,  1.90s/it]


 95%|█████████▍| 1197/1261 [38:28<02:01,  1.90s/it]


 95%|█████████▌| 1198/1261 [38:30<01:59,  1.90s/it]


 95%|█████████▌| 1199/1261 [38:32<01:58,  1.91s/it]


 95%|█████████▌| 1200/1261 [38:33<01:56,  1.91s/it]


 95%|█████████▌| 1201/1261 [38:35<01:55,  1.93s/it]


 95%|█████████▌| 1202/1261 [38:38<01:56,  1.98s/it]


 95%|█████████▌| 1203/1261 [38:40<01:56,  2.01s/it]


 95%|█████████▌| 1204/1261 [38:41<01:52,  1.97s/it]


 96%|█████████▌| 1205/1261 [

[MoviePy] Done.
[MoviePy] >>>> Video ready: video_output/project_video.mp4 



In [87]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_out))